In [1]:
import pandas as pd
import numpy as np
import pyodbc
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
from scipy.interpolate import interp1d

import requests
import folium

In [2]:
# Configuración de los parámetros de conexión
server = 'localhost\\SQLEXPRESS'  # O el punto '.' que usaste en SSMS
database = 'EM_CUNBOY'

# Cadena de conexión usando Autenticación de Windows (Trusted_Connection)
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;'
)


In [3]:
query = "select * from dbo.estaciones_full;"
conn = pyodbc.connect(conn_str)
raw_data = pd.read_sql(query, conn)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_15788\341712462.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_data = pd.read_sql(query, conn)


In [4]:
query = "select * from dbo.coordenadas_estaciones;"
conn = pyodbc.connect(conn_str)
coor = pd.read_sql(query, conn)
coor.drop(["tx_id"], axis=1, inplace=True)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_15788\2772575436.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coor = pd.read_sql(query, conn)


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80635 entries, 0 to 80634
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   tx_id              80635 non-null  int64         
 1   codigo_estacion    80635 non-null  object        
 2   fecha_observacion  80635 non-null  datetime64[ns]
 3   sensor_pre         80635 non-null  object        
 4   sensor_vel         80635 non-null  object        
 5   sensor_dir         80635 non-null  object        
 6   sensor_tem         80635 non-null  object        
 7   presion            80635 non-null  float64       
 8   velocidad          80635 non-null  float64       
 9   direccion          80635 non-null  float64       
 10  temperatura        80635 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 6.8+ MB


# explo

## coordenadas

In [6]:
coor.apply(lambda df: f"{df["codigo_estacion"]}: ( {df["latitud"]} , {df["longitud"]} )", axis=1)

0     0035085080: ( 5.099 , -73.436 )
1     0035080050: ( 5.096 , -73.053 )
2     0024015300: ( 5.643 , -73.518 )
3     0035025060: ( 4.587 , -73.996 )
4     0024035503: ( 5.818 , -72.844 )
                   ...               
79     0021206980: ( 4.785 , -73.87 )
80    0023125160: ( 5.647 , -74.071 )
81    2401500080: ( 5.634 , -73.386 )
82    0021205660: ( 4.579 , -74.523 )
83    2403500084: ( 5.758 , -72.795 )
Length: 84, dtype: object

## registro fechas

In [7]:
# raw_data.groupby(["codigo_estacion", "fecha_observacion"])[["presion", "velocidad", "direccion", "temperatura"]].mean()

In [7]:
df = raw_data.copy()
df.drop("tx_id", axis=1, inplace=True)

In [8]:
print(df.shape)
df = df.groupby(["codigo_estacion", "fecha_observacion"])[["presion", "velocidad", "direccion", "temperatura"]].mean().reset_index()
print(df.shape)

(80635, 10)
(71991, 6)


In [9]:
df

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura
0,0021195120,2025-12-01 00:00:00,855.3,1.3,301.0,19.4
1,0021195120,2025-12-01 01:00:00,856.3,1.3,23.0,18.9
2,0021195120,2025-12-01 02:00:00,857.1,2.3,32.0,18.6
3,0021195120,2025-12-01 03:00:00,857.4,2.5,30.0,18.4
4,0021195120,2025-12-01 04:00:00,857.8,1.0,356.0,18.0
...,...,...,...,...,...,...
71986,3507500133,2025-12-31 19:00:00,691.6,3.6,66.0,9.5
71987,3507500133,2025-12-31 20:00:00,691.4,2.8,70.0,9.1
71988,3507500133,2025-12-31 21:00:00,691.4,2.4,200.0,9.6
71989,3507500133,2025-12-31 22:00:00,691.6,1.4,226.0,9.8


In [11]:
# print(df.shape)
# print("-"*50)
# cols = ["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"]
# for col in cols:
#     print(df[col].value_counts())
#     col_0 = df[col].value_counts().index[0]
#     df = df[df[col] == col_0].copy()
#     print("-"*50)
# print(df.shape)

In [10]:
estaciones = df["codigo_estacion"].unique()

In [11]:
# df = df.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)
df = df.sort_values(["fecha_observacion", "codigo_estacion"]).reset_index(drop=True)

In [12]:
df["fecha_observacion"].diff().unique()

<TimedeltaArray>
[NaT, '0 days 00:00:00', '0 days 00:10:00']
Length: 3, dtype: timedelta64[ns]

In [15]:
# fecha_minima = pd.to_datetime("2025-12-15")
# df = df[df["fecha_observacion"] >= fecha_minima]

In [16]:
estaciones_ok = list()
n = 1
for estacion in estaciones:
    min_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].min()
    max_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].max()
    diff_temp = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].diff().unique()
    rows_temp = df.loc[df["codigo_estacion"] == estacion].shape[0]
    # if len(diff_temp) == 2: 
    # if rows_temp == 744 and not estacion in ["0024035250", "0024035360"]:
    if rows_temp == 744:
    # if rows_temp == 408:
    # if min_date <= pd.to_datetime("2025-12-14") and len(diff_temp) == 2:
    # if rows_temp == 408 and not estacion in ["0024035250"]:
    # 0021205509
    # if True:
    # if not estacion in ["0021205509"]:
        print(n , estacion)
        estaciones_ok.append(estacion)
        for diff in diff_temp: print(">", diff)
        print("cantidad de registros: ", rows_temp)
        print("fecha min: ", min_date)
        print("fecha max: ", max_date)
        print("-"*50)
        n += 1

1 0021195120
> NaT
> 0 days 01:00:00
cantidad de registros:  744
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
2 0021195170
> NaT
> 0 days 01:00:00
cantidad de registros:  744
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
3 0021205660
> NaT
> 0 days 01:00:00
cantidad de registros:  744
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
4 0021205670
> NaT
> 0 days 01:00:00
cantidad de registros:  744
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
5 0021205940
> NaT
> 0 days 01:00:00
cantidad de registros:  744
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
6 0023125170
> NaT
> 0 days 01:00:00
cantidad de registros:  744
fecha min:  2025-12-01 00:00:00
fec

# Elevación

In [17]:
def obtener_alturas_batch(coordenadas):
    """
    Consulta la elevación de múltiples puntos usando la API de Open-Elevation.
    coordenadas: Lista de diccionarios [{'latitude': lat, 'longitude': lon}, ...]
    """
    url = "https://api.open-elevation.com/api/v1/lookup"
    payload = {"locations": coordenadas}
    
    try:
        response = requests.post(url, json=payload, timeout=20)
        if response.status_code == 200:
            return response.json()['results']
    except Exception as e:
        print(f"Error al consultar la API: {e}")
    return None

In [18]:
puntos = list()
for idx in range(len(coor)):
    puntos.append({"latitude": coor.loc[idx, "latitud"], "longitude": coor.loc[idx, "longitud"]})
    ...

In [19]:
resultados = obtener_alturas_batch(puntos)

In [20]:
resultados_dics = {"latitud":list(), "longitud":list(), "altura":list()}
resultados

for resultado in resultados:
    resultados_dics["latitud"].append(resultado["latitude"])
    resultados_dics["longitud"].append(resultado["longitude"])
    resultados_dics["altura"].append(resultado["elevation"])

In [21]:
coor_alt = pd.DataFrame(resultados_dics)

In [22]:
print(f"rows: {df.shape[0]:,}")
wsdata = df.loc[df["codigo_estacion"].isin(estaciones_ok)].copy()
# wsdata.drop(["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"], axis=1, inplace=True)
print(f"rows: {wsdata.shape[0]:,}")
print(f"{100 * wsdata.shape[0] / df.shape[0]:,.2f} %")

rows: 71,991
rows: 12,648
17.57 %


# WSDATA

In [23]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura
0,0021195120,2025-12-01,855.3,1.3,301.0,19.4
1,0021195170,2025-12-01,718.9,2.5,141.0,12.9
11,0021205660,2025-12-01,922.4,0.8,209.0,23.9
12,0021205670,2025-12-01,809.1,1.0,47.0,17.2
15,0021205940,2025-12-01,746.6,0.6,241.0,10.3


In [24]:
wsdata = wsdata.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [25]:
wsdata["longitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["longitud"])))
wsdata["latitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["latitud"])))

In [26]:
wsdata = pd.merge(
    left=wsdata,
    right=coor_alt,
    how="left",
    on=["latitud", "longitud"]
)

In [27]:
wsdata.drop_duplicates(inplace=True)
wsdata.reset_index(drop=True, inplace=True)

In [28]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura,longitud,latitud,altura
0,0021195120,2025-12-01 00:00:00,855.3,1.3,301.0,19.4,-74.393,4.393,1423.0
1,0021195120,2025-12-01 01:00:00,856.3,1.3,23.0,18.9,-74.393,4.393,1423.0
2,0021195120,2025-12-01 02:00:00,857.1,2.3,32.0,18.6,-74.393,4.393,1423.0
3,0021195120,2025-12-01 03:00:00,857.4,2.5,30.0,18.4,-74.393,4.393,1423.0
4,0021195120,2025-12-01 04:00:00,857.8,1.0,356.0,18.0,-74.393,4.393,1423.0


In [29]:
wsdata["fecha_observacion"].sort_values().diff().value_counts()

fecha_observacion
0 days 00:00:00    11904
0 days 01:00:00      743
Name: count, dtype: int64

In [30]:
fecha_inicio = wsdata["fecha_observacion"].min()
fecha_fin = wsdata["fecha_observacion"].max()
print(f"{fecha_inicio=} , {fecha_fin=}")
rango_tiempo = pd.date_range(fecha_inicio, fecha_fin, freq="10min").to_list()
estaciones_registradas = wsdata["codigo_estacion"].unique()
len(estaciones_registradas)


fecha_inicio=Timestamp('2025-12-01 00:00:00') , fecha_fin=Timestamp('2025-12-31 23:00:00')


17

# Completar fechas con NaN

In [31]:
estaciones_registradas = wsdata["codigo_estacion"].unique()
estaciones = list()
latitudes = list()
longitudes = list()
alturas = list()

for estacion in estaciones_registradas:
    estaciones += [estacion]*len(rango_tiempo)
    latitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "latitud"].unique()[0]]*len(rango_tiempo)
    longitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "longitud"].unique()[0]]*len(rango_tiempo)
    alturas += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "altura"].unique()[0]]*len(rango_tiempo)
    
    ...

fechas = rango_tiempo * len(estaciones_registradas)

In [32]:
base_estacion_fechas = pd.DataFrame({"codigo_estacion":estaciones, "fecha_observacion":fechas, "latitud":latitudes, "longitud":longitudes, "altura":alturas})
wsdata_full = pd.merge(left=base_estacion_fechas, right=wsdata, on=["codigo_estacion", "fecha_observacion", "latitud", "longitud", "altura"], how="left")

In [33]:
wsdata_full.head()

,codigo_estacion,fecha_observacion,latitud,longitud,altura,presion,velocidad,direccion,temperatura
0,0021205012,2025-12-01 00:00:00,4.637,-74.089,2554.0,751.7,1.2,69.6,14.2
1,0021205012,2025-12-01 00:10:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN
2,0021205012,2025-12-01 00:20:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN
3,0021205012,2025-12-01 00:30:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN
4,0021205012,2025-12-01 00:40:00,4.637,-74.089,2554.0,NaN,NaN,NaN,NaN


In [34]:
wsdata_full = wsdata_full.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [35]:
wsdata_full["vel_u"] = wsdata_full.apply(lambda df: df["velocidad"] * np.sin(np.pi*(df["direccion"]/180)), axis=1)
wsdata_full["vel_v"] = wsdata_full.apply(lambda df: df["velocidad"] * np.cos(np.pi*(df["direccion"]/180)), axis=1)

In [36]:
wsdata_full["segundos"] = (wsdata_full["fecha_observacion"] - fecha_inicio) / np.timedelta64(1, 's')

In [37]:
wsdata_full = wsdata_full[[
    'codigo_estacion',
    'latitud',
    'longitud',
    'altura',
    'segundos',
    'presion',
    'vel_u',
    'vel_v',
    'temperatura',
]].copy()

In [38]:
wsdata_full.sample(5)

,codigo_estacion,latitud,longitud,altura,segundos,presion,vel_u,vel_v,temperatura
4662,0021205710,4.668,-74.101,2552.0,118800.0,750.1,0.034878,-0.498782,11.1
7812,0021205710,4.668,-74.101,2552.0,2008800.0,751.1,-0.497423,-0.335516,12.5
14629,0021206560,4.625,-74.156,2550.0,742200.0,NaN,NaN,NaN,NaN
29228,2120500204,4.622,-74.103,2558.0,1466400.0,NaN,NaN,NaN,NaN
18247,0021206600,4.787,-74.102,2553.0,234600.0,NaN,NaN,NaN,NaN


In [ ]:
# path_save_wsdata = Path().cwd().parent / "data" / "raw" / "em_bgta_251201_251231.parquet"
# path_save_wsdata_full = Path().cwd().parent / "data" / "raw" / "em_bgta_251201_251231_full.parquet"
# wsdata.to_parquet(path_save_wsdata, engine="fastparquet", index=False)
# wsdata_full.to_parquet(path_save_wsdata_full, engine="fastparquet", index=False)

In [31]:
path_save_wsdata = Path().cwd().parent / "data" / "raw" / "em_cundinamarca_boyaca_251201_251231.parquet"
wsdata.to_parquet(path_save_wsdata, engine="fastparquet", index=False)
